# Temporal Collocation of irregularly sampled timeseries

Satellite observations usually have an irregular temporal sampling pattern (intervals between 6-36 hours), which is mostly controlled by the orbit of the satellite and the instrument measurement geometry. On the other hand, in-situ instruments or land surface models generally sample on regular time intervals (commonly every 1, 3, 6, 12 or 24 hours). 
In order to compute error/performance statistics (such as RMSD, bias, correlation) between the time series coming from different sources, it is required that observation pairs (or triplets, etc.) are found which (nearly) coincide in time.

A simple way to identify such pairs is by using a nearest neighbor search. First, one time series needs to be selected as temporal reference (i.e. all other time series will be matched to this reference) and second, a tolerance window (typically around 1-12 hours) has to be defined characterizing the temporal correlation of neighboring observation (i.e. observations outside of the tolerance window are no longer be considered as representative neighbors). 

## Temporal collocation in pytesmo

Pytesmo contains the function `pytesmo.temporal_matching.temporal_collocation` for temporally collocating timeseries. Currently, it implements nearest neighbour matching and a windowed mean. It requires a reference index (can also be a DataFrame or a Series), a DataFrame (or Series) to be collocated, and a window.

```
collocated = temporal_collocation(reference, input_frame, window)
```

The window argument corresponds to the time intervals that are included in the nearest neighbour search in each direction, e.g. if the reference time is $t$ and the window $\Delta$, the nearest neighbour inside $[t-\Delta, t+\Delta]$ is returned. If no neighbour is found `np.nan` is used as replacement. NaNs can be dropped from the returned dataframe by providing the optional keyword argument ``dropna=True`` to the function.

Below are two simple examples which demonstrate the usage. The first example assumes that the index of data to be collocated is shifted by 3 hours with respect to the reference, while using a 6 hour window. The second example uses an index that is randomly shifted by $\pm12$ hours with respect to the reference. The second example also uses a 6 hour window, which results in some missing values in the resulting dataframe.

In [ ]:
import numpy as np
import pandas as pd

from pytesmo.temporal_matching import temporal_collocation, combined_temporal_collocation

In [ ]:
# create reference time series
ref = pd.date_range("2020-01-01", "2020-12-31", freq="D")
# temporal_collocation can also take a DataFrame or Series as reference input,
# in case their index is a DatetimeIndex.

# create other time series as dataframe
values = np.random.randn(len(ref), 3)
shifted = pd.DataFrame(values, index=ref + pd.Timedelta(hours=3), 
                       columns=list(map(lambda x: f"shifted_{x}", range(3))))
random_shift = np.random.uniform(-12, 12, len(ref))
random = pd.DataFrame(values, index=ref + pd.to_timedelta(random_shift, "H"),
                      columns=list(map(lambda x: f"random_{x}", range(3))))

shifted.head()

In [ ]:
random.head()

We can now match the shifted timeseries to the reference index by using a 6-hour window, either for a nearest neighbour search, or for taking a windowed mean. Both should return unchanges timeseries, except for the index.

In [ ]:
# match the regularly shifted data
window = pd.Timedelta(hours=6)
matched_shifted_nn = temporal_collocation(ref, shifted, window, method="nearest")
matched_shifted_mean = temporal_collocation(ref, shifted, window, method="mean")

# the data should be the same before and after matching for both methods
assert np.all(shifted.values == matched_shifted_nn.values)
assert np.all(shifted.values == matched_shifted_mean.values)

matched_shifted_nn

We can do the same for the randomly shifted timeseries. Here we should see some changes, because sometimes there's no value inside the window that we are looking at. However, the result of mean and nearest neighbour should be the same

In [ ]:
# match the randomly shifted data
matched_random_nn = temporal_collocation(ref, random, window, method="nearest")
matched_random_mean = temporal_collocation(ref, random, window, method="mean")

# the data should be the same as before matching at the locations where the shift
# was below 6 hours, and should be np.nan when shift was larger
should_be_nan = np.abs(random_shift) > 6
assert np.all(matched_random_nn[~should_be_nan].values == random[~should_be_nan].values)
assert np.all(np.isnan(matched_random_nn[should_be_nan].values))
np.testing.assert_equal(matched_random_nn.values, matched_random_mean.values)
matched_random_nn

### Returning the original index

`temporal_collocation` can also return the original index of the data that was matched as a separate column in the resulting DataFrame, if required, and can additionally also calculate the distance to the reference. The column names are "index_other" and "distance_other", respectively.

In [ ]:
# also return original index and distance
matched_shifted = temporal_collocation(
    ref, shifted, window, 
    method="nearest", 
    return_index=True, 
    return_distance=True
)

# the index should be the same as unmatched, and the distance should be 3  hours
assert np.all(matched_shifted["index_other"].values == shifted.index.values)
assert np.all(matched_shifted["distance_other"] == pd.Timedelta(hours=3))

matched_shifted

## Flags

Satellite data often contains flags indicating quality issues with the data. With `temporal_collocation` it is possible to use this information. Flags can either be provided as array (of the same length as the input DataFrame), or the name of a column in the DataFrame to be used as flag can be provided as string. Any non-zero flag is interpreted as indicating invalid data. By default this will not be used, but when passing ``use_invalid=True``, the invalid values will be used in case no valid match was found.

For the following example, we reuse the input data shifted by 3 hours, but we will now assume that the first 3 observations had quality issues.

In [ ]:
# flag the first 3 observations as invalid
flag = np.zeros(len(ref), dtype=bool)
flag[0:3] = True
flag[0:10]

In [ ]:
matched_flagged = temporal_collocation(ref, shifted, window, flag=flag)

# the first 3 values should be NaN, otherwise the result should be the same as matched_shifted
assert np.all(np.isnan(matched_flagged.values[0:3, :]))
assert np.all(matched_flagged.values[3:, :] == matched_shifted.values[3:, 0:3])  # excluding additonal columns
matched_flagged

In [ ]:
# This also works when the flag is already in the input frame, but note that
# in the output frame the nonzero flag values are replaced by NaN
flagged = shifted.assign(my_flag=flag)
matched_flagged = temporal_collocation(ref, flagged, window, flag="my_flag")

# the first 3 values should be NaN, otherwise the result should be the same as matched_shifted
assert np.all(np.isnan(matched_flagged.iloc[0:3, 0:3].values))
assert np.all(matched_flagged.iloc[3:, 0:3].values == matched_shifted.values[3:, 0:3])  # excluding additonal columns
matched_flagged

## Combined collocation

It is also possible to match multiple timeseries together against a reference dataset using the function `pytesmo.temporal_matching.combined_temporal_collocation`. With the keyword argument `combined_dropna` it's possible to drop data where one of the input datasets has missing values from all other input datasets.

In [ ]:
combined_match = combined_temporal_collocation(ref, (random, shifted), window, combined_dropna=True)
# matched dataframe should have same length as matched_random_nn without NaNs
assert len(combined_match == len(matched_random_nn.dropna()))
combined_match